In [75]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [76]:

input_file = r'D:\School\ADMU\4Y\SEM 1\MATH 199.11\input_gwap.csv'
data = pd.read_csv(input_file)

data = data[['GWAP', 'FLOW_LUZ','RGWAP_DR','RGWAP_FR','RGWAP_RD','RGWAP_RU']]  # Select the relevant time series column


scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data.values)
data_scaler = MinMaxScaler()
columns = ['GWAP', 'FLOW_LUZ','RGWAP_DR','RGWAP_FR','RGWAP_RD','RGWAP_RU']
        
X  = data[columns].values
y = data['GWAP'].values.reshape(-1,1)


In [77]:
class Dataset(Dataset):
    
    def __init__(self, X, y, seq_len):
        self.X = torch.tensor(X).float()
        self.y = torch.tensor(y).float()
        self.seq_len = seq_len
    
    def __len__(self):
        return self.X.__len__() - (self.seq_len - 1)
        
    def __getitem__(self, idx):
        return (self.X[idx:idx+self.seq_len-1], self.y[idx+self.seq_len-1])

In [103]:
X_test, X_tmp, y_test, y_tmp = train_test_split(X, y, test_size=.8)

X_val, X_train, y_val, y_train = train_test_split(X_tmp, y_tmp, test_size=.75)
seq_len=1440
batch_size=64

train_dataset = Dataset(X_train, y_train, seq_len)
train_dataloader = DataLoader(train_dataset, batch_size, shuffle=False)

val_dataset = Dataset(X_val, y_val, seq_len)    
val_dataloader = DataLoader(val_dataset, batch_size, shuffle=False) 

test_dataset = Dataset(X_test, y_test, seq_len)
test_dataloader = DataLoader(test_dataset, batch_size, shuffle=False)


In [80]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # LSTM layer
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        # Fully connected layer
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # Initialize hidden and cell states
        h0 = torch.zeros(self.num_layers,  x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
    
        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])  # Take the output from the last time step
        return out

In [109]:
# Define model parameters
input_size = X_train.shape[1]  # Number of features
hidden_size = 64
output_size = y_train.shape[1]  # Number of output features
num_layers = 2
model = LSTMModel(input_size, hidden_size, output_size, num_layers).to(device)
criterion=nn.MSELoss()

In [125]:
def train(model,train_dataloader,device,optimizer):
    model.train()


    for i, (inputs, target) in enumerate(train_dataloader):
        inputs, target = inputs.to(device), target.to(device)
        optimizer.zero_grad()
        torch.cuda.empty_cache()
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
        
        
    return loss.item()

In [126]:
@torch.no_grad()
def evaluate(model, test_dataloader, device):
    model.eval()

    for i, (inputs, target) in enumerate(train_dataloader):
        inputs, target = inputs.to(device), target.to(device)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, target)
        
    
    return loss.item()


In [127]:

def run(model, train_dataloader, test_dataloader, device, epoch):
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    
    for epoch in range(epoch):
        loss = train(model, train_dataloader, device, optimizer)
        loss = evaluate(model, train_dataloader, device)
        test_loss = evaluate(model, test_dataloader, device)
        print(epoch, loss, test_loss)
        if (epoch + 1) == 100 or (epoch + 1) % 20 == 0:
            print(f'Epoch {epoch+1:04d} | loss: {loss:.4f} '
                f'test_loss: {test_loss:.4f} ')
    
        if loss < 1e-3 and test_loss < 1e-3:
            break

In [128]:
epoch=100
run(model, train_dataloader, test_dataloader, device, epoch)

0 745987200.0 227505136.0


KeyboardInterrupt: 